In [1]:
!pip install autoawq
!pip install nvidia-ml-py3

In [2]:
from awq import AutoAWQForCausalLM

In [3]:
from transformers import AutoTokenizer

In [4]:
from huggingface_hub import notebook_login

In [5]:
notebook_login()

In [8]:
model_path = "meta-llama/Llama-3.2-3B"
quant_path = "Llama-3.2-3B-AWQ-4bit"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit":4}

In [9]:
#load the model and tokenizer
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/41.2k [00:00<?, ?B/s]

...

params.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
model.quantize(tokenizer, quant_config=quant_config)

README.md:   0%|          | 0.00/167 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


val.jsonl.zst:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/214670 [00:00<?, ? examples/s]

AWQ: 100%|██████████| 28/28 [48:51<00:00, 104.70s/it]


In [11]:
#save the quantized model
model.save_quantized("./"+quant_path, safetensors=True)
tokenizer.save_pretrained("./"+quant_path)

('./Llama-3.2-3B-AWQ-4bit/tokenizer_config.json',
 './Llama-3.2-3B-AWQ-4bit/special_tokens_map.json',
 './Llama-3.2-3B-AWQ-4bit/tokenizer.json')

In [12]:
from huggingface_hub import HfApi

In [13]:
username = "Moksh3110"
MODEL_NAME = quant_path

In [ ]:
api = HfApi(token="your_token")

In [17]:
api.create_repo(
    repo_id = f"{username}/{MODEL_NAME}",
    repo_type="model"
)

RepoUrl('https://huggingface.co/Moksh3110/Llama-3.2-3B-AWQ-4bit', endpoint='https://huggingface.co', repo_type='model', repo_id='Moksh3110/Llama-3.2-3B-AWQ-4bit')

In [20]:
api.upload_folder(
    repo_id = f"{username}/{MODEL_NAME}",
    folder_path = "/content/Llama-3.2-3B-AWQ-4bit"
)

model.safetensors:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Moksh3110/Llama-3.2-3B-AWQ-4bit/commit/3f4f082832a6e9a482a53f500315521f7af2e92e', commit_message='Upload folder using huggingface_hub', commit_description='', oid='3f4f082832a6e9a482a53f500315521f7af2e92e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Moksh3110/Llama-3.2-3B-AWQ-4bit', endpoint='https://huggingface.co', repo_type='model', repo_id='Moksh3110/Llama-3.2-3B-AWQ-4bit'), pr_revision=None, pr_num=None)

In [2]:
from pynvml import *

In [3]:
import torch, time

In [4]:
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("Moksh3110/Llama-3.2-3B-AWQ-4bit", torch_dtype=torch.float16, device_map={"":0})
print(print_gpu_utilization())

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model.safetensors:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

GPU memory occupied: 2537 MB.
None


In [6]:
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
else:
  compute_dtype = torch.float16

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", torch_dtype=compute_dtype, device_map={"":0})
print(print_gpu_utilization())

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

GPU memory occupied: 21701 MB.
None
